In [2]:
# Instala o PySpark (necessário apenas em ambientes como Google Colab)
!pip install pyspark

In [3]:
# Importações principais do PySpark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, when
from pyspark.sql.functions import to_date, year
from pyspark.sql.types import IntegerType

In [4]:
# Inicia a sessão do Spark
spark = SparkSession.builder.getOrCreate()

In [5]:
# Leitura do arquivo CSV com estatísticas de vídeos
df_video = spark.read.csv(
    '/content/drive/MyDrive/data/Material/videos-stats.csv',
    header=True,
    inferSchema=True
)

In [6]:
# Preenche valores nulos nas colunas Likes, Comments e Views com 0
df_video = df_video.fillna({'Likes': 0, 'Comments': 0, 'Views': 0})

In [7]:
# Leitura do arquivo CSV com comentários
df_comments = spark.read.csv(
    '/content/drive/MyDrive/data/Material/comments.csv',
    header=True,
    inferSchema=True
)


In [9]:
# Exibe a quantidade de registros antes da limpeza
print(f'Quantidade de registros de vídeos: {df_video.count()}')
print(f'Quantidade de registros de comentários: {df_comments.count()}')

Quantidade de registros de vídeos: 1881
Quantidade de registros de comentários: 30036


In [10]:
# Remove registros com 'Video ID' nulo
df_video = df_video.filter(df_video['Video ID'].isNotNull())
df_comments = df_comments.filter(df_comments['Video ID'].isNotNull())

# Exibe a quantidade de registros após remoção dos nulos
print(f'Quantidade de registros de vídeos: {df_video.count()}')
print(f'Quantidade de registros de comentários: {df_comments.count()}')

Quantidade de registros de vídeos: 1881
Quantidade de registros de comentários: 22555


In [11]:
# Remove vídeos com 'Video ID' duplicados
df_video = df_video.dropDuplicates(['Video ID'])

In [12]:
# Converte as colunas Likes, Comments e Views para o tipo inteiro
df_video = df_video.withColumn("Likes", df_video["Likes"].cast(IntegerType())) \
                   .withColumn("Comments", df_video["Comments"].cast(IntegerType())) \
                   .withColumn("Views", df_video["Views"].cast(IntegerType()))

In [13]:
# Cria a coluna 'Interaction' com a soma de Likes, Comments e Views
df_video = df_video.withColumn(
    'Interaction',
    col('Likes') + col('Comments') + col('Views')
)

In [14]:
# Converte a coluna 'Published At' para o tipo data (esperando formato 'dd-MM-yyyy')
df_video = df_video.withColumn('Published At', to_date(col('Published At'), 'dd-MM-yyyy'))

In [15]:
# Cria a coluna 'Year' extraída da data de publicação
df_video = df_video.withColumn('Year', year(col('Published At')))

In [16]:
# Junta os dados de vídeos com os comentários pelo campo 'Video ID'
df_join_video_comments = df_video.join(df_comments, on='Video ID', how='inner')
df_join_video_comments.show()

+-----------+---+--------------------+------------+-------+-----+--------+-------+-----------+----+---+--------------------+------+---------+
|   Video ID|_c0|               Title|Published At|Keyword|Likes|Comments|  Views|Interaction|Year|_c0|             Comment| Likes|Sentiment|
+-----------+---+--------------------+------------+-------+-----+--------+-------+-----------+----+---+--------------------+------+---------+
|wAZZ-UWGVHI|  0|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|  0|Let's not forget ...|  95.0|      1.0|
|wAZZ-UWGVHI|  0|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|  1|Here in NZ 50% of...|  19.0|      0.0|
|wAZZ-UWGVHI|  0|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|  2|I will forever ac...| 161.0|      2.0|
|wAZZ-UWGVHI|  0|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|  3|Whenever I go to ...|   8.0|      0.0|
|wAZZ-

In [17]:
# Lê outro conjunto de dados com vídeos dos EUA
df_us_video = spark.read.csv(
    '/content/drive/MyDrive/data/Material/USvideos.csv',
    header=True,
    inferSchema=True
)

In [18]:
# Junta os vídeos com os vídeos dos EUA com base no título (Title)
df_join_video_us = df_video.join(df_us_video, on='Title', how='inner')
df_join_video_us.show()


+--------------------+----+-----------+------------+----------------+------+--------+---------+-----------+----+-----------+-------------+-----------------+-----------+--------------------+--------------------+--------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|               Title| _c0|   Video ID|Published At|         Keyword| Likes|Comments|    Views|Interaction|Year|   video_id|trending_date|    channel_title|category_id|        publish_time|                tags|   views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+--------------------+----+-----------+------------+----------------+------+--------+---------+-----------+----+-----------+-------------+-----------------+-----------+--------------------+--------------------+--------+------+--------+-------------+--------------------+-----------------+------------

In [19]:
# Verifica a quantidade de valores NÃO nulos em cada coluna do df_video
df_video.select([
    _sum(when(col(c).isNotNull(), 1).otherwise(0)).alias(c)
    for c in df_video.columns
]).show()

+----+-----+--------+------------+-------+-----+--------+-----+-----------+----+
| _c0|Title|Video ID|Published At|Keyword|Likes|Comments|Views|Interaction|Year|
+----+-----+--------+------------+-------+-----+--------+-----+-----------+----+
|1869| 1869|    1869|        1869|   1869| 1869|    1869| 1869|       1869|1869|
+----+-----+--------+------------+-------+-----+--------+-----+-----------+----+



In [20]:
# Remove a coluna '_c0' do df_video
df_video = df_video.drop('_c0')

# Salva df_video no formato Parquet, com cabeçalho (esquema é incluído automaticamente)
df_video.write.mode('overwrite').option('header', True).parquet('/content/videos-tratados-parquet')

In [21]:
# Remove a coluna '_c0' do df_join_video_comments (caso exista)
df_join_video_comments = df_join_video_comments.drop('_c0')

# Salva o dataframe combinado de vídeos e comentários em Parquet
df_join_video_comments.write.mode('overwrite').option('header', True).parquet('/content/videos-comments-tratados-parquet')

AnalysisException: [COLUMN_ALREADY_EXISTS] The column `likes` already exists. Consider to choose another name or rename the existing column.